In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!unzip /kaggle/input/sberbank-russian-housing-market/train.csv.zip

In [ ]:
x_train = pd.read_csv(r'../input/sberbank-russian-housing-market-cleaned-data/train_data.csv')
y_train = pd.read_csv(r'../input/sberbank-russian-housing-market-cleaned-data/ytrain_data.csv')
x_test = pd.read_csv(r'../input/sberbank-russian-housing-market-cleaned-data/test_data.csv')
y_test = pd.read_csv(r'../input/sberbank-russian-housing-market-cleaned-data/ytest_data.csv')

In [ ]:
x_test.head()

In [ ]:
x_train.drop('Unnamed: 0',axis=1,inplace=True)
x_test.drop('Unnamed: 0',axis=1,inplace=True)
y_train.drop('Unnamed: 0',axis=1,inplace=True)
y_test.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
train_data = scaler.fit_transform(x_train)
test_data = scaler.fit_transform(x_test)

In [ ]:
train = pd.DataFrame(train_data,columns=x_train.columns)
test = pd.DataFrame(test_data,columns=x_test.columns)

In [ ]:
train.shape

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        model.add(layers.Dense(units= hp.Int('units_'+str(i),
                                        min_value=32,
                                        max_value =512,
                                        step=32),
                                        activation='relu'))
        model.add(layers.Dense(1,activation='linear'))
        model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4,1e-5])),
        loss ='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [ ]:
tuner = RandomSearch(build_model,objective='val_mean_absolute_error',max_trials=5,executions_per_trial=3,
directory='project1.1',project_name='Russiann Data Price')

In [ ]:
tuner.search_space_summary

In [ ]:
tuner.search(train,y_train,epochs=10,validation_data = (test,y_test))

In [ ]:
tuner.results_summary()

In [ ]:
from tensorflow.keras.layers import Dense, Activation
import tensorflow

In [ ]:
callback = tensorflow.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=2,
    verbose=1,
    mode="min",
    baseline=None,
    restore_best_weights=False,
)

In [ ]:
model = keras.Sequential()
model.add(Dense(units=416,activation='relu',input_dim=50,kernel_initializer='he_uniform'))
model.add(Dense(units=480,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=64,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=128,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=416,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=64,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=192,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=512,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=384,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=288,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=96,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=448,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=288,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=256,activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(units=384,activation='relu',kernel_initializer='he_uniform'))
model.add(layers.Dense(1,activation='linear'))
model.compile(optimizer=keras.optimizers.Adam(learning_rate= 0.01),
        loss ='mean_absolute_error',
        metrics=['mean_absolute_error'])

In [ ]:
model_history = model.fit(train,y_train,batch_size = 200,epochs=100,validation_data=(test,y_test),callbacks=[callback])

In [ ]:
y_pred = model.predict(test)

In [ ]:
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test,y_pred))

In [ ]:
model.save("House_price.h5")